In [19]:
import pymssql
import json
import os
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv


In [20]:
load_dotenv()

conn = pymssql.connect(
    server=os.getenv('DB_SERVER'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    database=os.getenv('DB_NAME'),
    port=os.getenv('DB_PORT')
)

cursor = conn.cursor()
print("✅ Conectado")


✅ Conectado


In [21]:
now = datetime.now()
print("=" * 80)
print(f"🕐 VERIFICACIÓN DE PREDICCIONES")
print("=" * 80)
print(f"📅 Hora actual del sistema: {now.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⏰ Día de la semana: {now.strftime('%A')}")
print("=" * 80)

cursor.execute("SELECT COUNT(*) FROM Forecast")
total = cursor.fetchone()[0]
print(f"\n📊 Total de registros en tabla: {total}")

cursor.execute("SELECT COUNT(DISTINCT municipio) FROM Forecast")
municipios = cursor.fetchone()[0]
print(f"📍 Municipios con predicciones: {municipios}")


🕐 VERIFICACIÓN DE PREDICCIONES
📅 Hora actual del sistema: 2026-01-05 09:46:45
⏰ Día de la semana: Monday

📊 Total de registros en tabla: 26
📍 Municipios con predicciones: 13


In [22]:
print("\n" + "=" * 80)
print("⏱️  ÚLTIMA ACTUALIZACIÓN")
print("=" * 80)

query = """
SELECT TOP 1
    created_at,
    COUNT(*) as total
FROM Forecast
GROUP BY created_at
ORDER BY created_at DESC
"""

cursor.execute(query)
row = cursor.fetchone()

if row:
    last_update = row[0]
    count = row[1]
    time_diff = now - last_update
    
    hours = int(time_diff.total_seconds() // 3600)
    minutes = int((time_diff.total_seconds() % 3600) // 60)
    
    print(f"📅 Última actualización: {last_update.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"⏱️  Hace: {hours} horas y {minutes} minutos")
    print(f"📊 Registros en esa actualización: {count}")
    
    if time_diff < timedelta(hours=2):
        print(f"✅ Las predicciones están ACTUALIZADAS (< 2 horas)")
    elif time_diff < timedelta(hours=6):
        print(f"⚠️  Las predicciones tienen cierta antigüedad (2-6 horas)")
    else:
        print(f"❌ Las predicciones están DESACTUALIZADAS (> 6 horas)")
else:
    print("❌ No hay predicciones en la base de datos")



⏱️  ÚLTIMA ACTUALIZACIÓN
📅 Última actualización: 2026-01-05 14:45:49
⏱️  Hace: -5 horas y 0 minutos
📊 Registros en esa actualización: 1
✅ Las predicciones están ACTUALIZADAS (< 2 horas)


In [23]:
print("\n" + "=" * 80)
print("📊 PREDICCIONES MÁS RECIENTES (TOP 5)")
print("=" * 80)

query = """
SELECT TOP 5
    municipio,
    datetime_inicio,
    wind_speed_input,
    wind_speed_output,
    created_at
FROM Forecast
ORDER BY created_at DESC
"""

cursor.execute(query)
rows = cursor.fetchall()

for idx, row in enumerate(rows, 1):
    mun = row[0]
    dt_inicio = row[1]
    input_array = json.loads(row[2])
    output_array = json.loads(row[3])
    created = row[4]
    
    time_since = now - created
    minutes_ago = int(time_since.total_seconds() / 60)
    
    print(f"\n{idx}. {mun.upper()}")
    print(f"   📅 Inicio predicción: {dt_inicio.strftime('%Y-%m-%d %H:%M')}")
    print(f"   🕐 Creado: {created.strftime('%Y-%m-%d %H:%M:%S')} (hace {minutes_ago} min)")
    print(f"   📥 Input: {len(input_array)} valores históricos")
    print(f"   📤 Output: {len(output_array)} horas predichas")
    print(f"   🌬️  Viento promedio: {sum(output_array)/len(output_array):.2f} m/s")
    print(f"   ⬆️  Máximo: {max(output_array):.2f} m/s")
    print(f"   ⬇️  Mínimo: {min(output_array):.2f} m/s")



📊 PREDICCIONES MÁS RECIENTES (TOP 5)

1. URIBIA
   📅 Inicio predicción: 2026-01-06 00:00
   🕐 Creado: 2026-01-05 14:45:49 (hace -299 min)
   📥 Input: 48 valores históricos
   📤 Output: 24 horas predichas
   🌬️  Viento promedio: 9.16 m/s
   ⬆️  Máximo: 15.07 m/s
   ⬇️  Mínimo: 4.36 m/s

2. SAN_JUAN_DEL_CESAR
   📅 Inicio predicción: 2026-01-06 00:00
   🕐 Creado: 2026-01-05 14:45:49 (hace -299 min)
   📥 Input: 48 valores históricos
   📤 Output: 24 horas predichas
   🌬️  Viento promedio: 3.18 m/s
   ⬆️  Máximo: 6.10 m/s
   ⬇️  Mínimo: 1.43 m/s

3. RIOHACHA
   📅 Inicio predicción: 2026-01-06 00:00
   🕐 Creado: 2026-01-05 14:45:49 (hace -299 min)
   📥 Input: 48 valores históricos
   📤 Output: 24 horas predichas
   🌬️  Viento promedio: 11.78 m/s
   ⬆️  Máximo: 19.42 m/s
   ⬇️  Mínimo: 6.16 m/s

4. MINGUEO
   📅 Inicio predicción: 2026-01-06 00:00
   🕐 Creado: 2026-01-05 14:45:49 (hace -299 min)
   📥 Input: 48 valores históricos
   📤 Output: 24 horas predichas
   🌬️  Viento promedio: 4.07 m/s


In [24]:
print("\n" + "=" * 80)
print("📈 DETALLE DE PREDICCIÓN MÁS RECIENTE")
print("=" * 80)

query = """
SELECT TOP 1
    municipio,
    datetime_inicio,
    wind_speed_input,
    wind_speed_output,
    created_at
FROM Forecast
ORDER BY created_at DESC
"""

cursor.execute(query)
row = cursor.fetchone()

if row:
    mun = row[0]
    dt_inicio = row[1]
    input_array = json.loads(row[2])
    output_array = json.loads(row[3])
    created = row[4]
    
    print(f"\n🏙️  Municipio: {mun.upper()}")
    print(f"📅 Predicción inicia: {dt_inicio.strftime('%Y-%m-%d %H:%M')}")
    print(f"🕐 Generada: {created.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📊 Datos de entrada: {len(input_array)} valores históricos")
    print(f"📊 Predicciones generadas: {len(output_array)} horas")
    
    print(f"\n🌬️  RESUMEN DE PREDICCIÓN:")
    print(f"   • Promedio: {sum(output_array)/len(output_array):.2f} m/s")
    print(f"   • Máximo: {max(output_array):.2f} m/s (hora {output_array.index(max(output_array))})")
    print(f"   • Mínimo: {min(output_array):.2f} m/s (hora {output_array.index(min(output_array))})")
    
    print(f"\n📥 ÚLTIMOS 5 VALORES DE ENTRADA:")
    for i, val in enumerate(input_array[-5:], start=44):
        print(f"   [{i}] {val:.2f} m/s")
    
    print(f"\n📤 PRIMERAS 5 PREDICCIONES:")
    for i, val in enumerate(output_array[:5]):
        pred_time = dt_inicio + timedelta(hours=i)
        print(f"   [{i}] {pred_time.strftime('%H:%M')} → {val:.2f} m/s")
    
    print(f"\n📤 ÚLTIMAS 5 PREDICCIONES:")
    for i, val in enumerate(output_array[-5:], start=len(output_array)-5):
        pred_time = dt_inicio + timedelta(hours=i)
        print(f"   [{i}] {pred_time.strftime('%H:%M')} → {val:.2f} m/s")
else:
    print("❌ No hay datos")



📈 DETALLE DE PREDICCIÓN MÁS RECIENTE

🏙️  Municipio: URIBIA
📅 Predicción inicia: 2026-01-06 00:00
🕐 Generada: 2026-01-05 14:45:49
📊 Datos de entrada: 48 valores históricos
📊 Predicciones generadas: 24 horas

🌬️  RESUMEN DE PREDICCIÓN:
   • Promedio: 9.16 m/s
   • Máximo: 15.07 m/s (hora 11)
   • Mínimo: 4.36 m/s (hora 2)

📥 ÚLTIMOS 5 VALORES DE ENTRADA:
   [44] 3.80 m/s
   [45] 6.30 m/s
   [46] 6.10 m/s
   [47] 4.70 m/s
   [48] 5.00 m/s

📤 PRIMERAS 5 PREDICCIONES:
   [0] 00:00 → 5.15 m/s
   [1] 01:00 → 4.80 m/s
   [2] 02:00 → 4.36 m/s
   [3] 03:00 → 4.51 m/s
   [4] 04:00 → 4.91 m/s

📤 ÚLTIMAS 5 PREDICCIONES:
   [19] 19:00 → 7.95 m/s
   [20] 20:00 → 8.36 m/s
   [21] 21:00 → 7.63 m/s
   [22] 22:00 → 7.42 m/s
   [23] 23:00 → 7.41 m/s


In [25]:
print("\n" + "=" * 80)
print("📋 RESUMEN POR MUNICIPIO")
print("=" * 80)

query = """
SELECT 
    municipio,
    datetime_inicio,
    created_at
FROM Forecast
ORDER BY municipio
"""

cursor.execute(query)
rows = cursor.fetchall()

print(f"\n{'Municipio':<20} {'Predicción desde':<20} {'Actualizado hace':<20}")
print("-" * 65)

for row in rows:
    mun = row[0]
    dt_inicio = row[1]
    created = row[2]
    time_diff = now - created
    minutes_ago = int(time_diff.total_seconds() / 60)
    
    if minutes_ago < 60:
        time_str = f"{minutes_ago} min"
    else:
        hours_ago = minutes_ago // 60
        time_str = f"{hours_ago}h {minutes_ago % 60}min"
    
    print(f"{mun:<20} {dt_inicio.strftime('%Y-%m-%d %H:%M'):<20} {time_str:<20}")



📋 RESUMEN POR MUNICIPIO

Municipio            Predicción desde     Actualizado hace    
-----------------------------------------------------------------
albania              2026-01-05 00:00     8h 36min            
albania              2026-01-06 00:00     -299 min            
barrancas            2026-01-06 00:00     -299 min            
barrancas            2026-01-05 00:00     8h 36min            
distraccion          2026-01-05 00:00     8h 36min            
distraccion          2026-01-06 00:00     -299 min            
el_molino            2026-01-06 00:00     -299 min            
el_molino            2026-01-05 00:00     8h 36min            
fonseca              2026-01-05 00:00     8h 36min            
fonseca              2026-01-06 00:00     -299 min            
hatonuevo            2026-01-06 00:00     -299 min            
hatonuevo            2026-01-05 00:00     8h 36min            
la_jagua_del_pilar   2026-01-05 00:00     8h 36min            
la_jagua_del_pilar   2026-

In [26]:
print("\n" + "=" * 80)
print("✅ RESUMEN FINAL")
print("=" * 80)

# Verificar coherencia
cursor.execute("""
    SELECT 
        COUNT(*) as total,
        COUNT(DISTINCT municipio) as municipios,
        COUNT(DISTINCT datetime_inicio) as fechas_inicio,
        MIN(created_at) as primera_actualizacion,
        MAX(created_at) as ultima_actualizacion
    FROM Forecast
""")

row = cursor.fetchone()
total, municipios, fechas, primera, ultima = row

print(f"\n📊 ESTADÍSTICAS:")
print(f"   • Total registros: {total}")
print(f"   • Municipios únicos: {municipios}")
print(f"   • Fechas de inicio únicas: {fechas}")

if primera and ultima:
    print(f"\n⏰ ACTUALIZACIONES:")
    print(f"   • Primera: {primera.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"   • Última: {ultima.strftime('%Y-%m-%d %H:%M:%S')}")
    
    time_diff = now - ultima
    hours_since = time_diff.total_seconds() / 3600
    
    print(f"\n🔄 ESTADO:")
    if hours_since < 1:
        print(f"   ✅ EXCELENTE - Actualizado hace {int(hours_since * 60)} minutos")
    elif hours_since < 2:
        print(f"   ✅ BUENO - Actualizado hace {hours_since:.1f} horas")
    elif hours_since < 6:
        print(f"   ⚠️  ACEPTABLE - Actualizado hace {hours_since:.1f} horas")
    else:
        print(f"   ❌ DESACTUALIZADO - Última actualización hace {hours_since:.1f} horas")
        print(f"   💡 Ejecuta: python scripts/update_db_simple.py --forecast-only")

print(f"\n💡 RECOMENDACIÓN:")
if municipios == 13 and total == 13:
    print("   ✅ Sistema funcionando correctamente")
    print("   ✅ 1 predicción por municipio (estructura correcta)")
elif total == 0:
    print("   ❌ No hay predicciones en la base de datos")
    print("   💡 Ejecuta: python scripts/update_db_simple.py --forecast-only")
else:
    print(f"   ⚠️  Estructura inconsistente")
    print(f"   💡 Se esperan 13 municipios, encontrados {municipios}")

print("\n" + "=" * 80)



✅ RESUMEN FINAL

📊 ESTADÍSTICAS:
   • Total registros: 26
   • Municipios únicos: 13
   • Fechas de inicio únicas: 2

⏰ ACTUALIZACIONES:
   • Primera: 2026-01-05 01:10:32
   • Última: 2026-01-05 14:45:49

🔄 ESTADO:
   ✅ EXCELENTE - Actualizado hace -299 minutos

💡 RECOMENDACIÓN:
   ⚠️  Estructura inconsistente
   💡 Se esperan 13 municipios, encontrados 13



In [27]:
cursor.close()
conn.close()
print("✅ Cerrado")


✅ Cerrado
